In [2]:
import time
%run simple_interval_propagation.ipynb

In [4]:
def model_3_property_check(output_min, output_max):
    if output_min >= 0.5:
        return True
    else:
        return False

In [11]:
# if the output of the interval propagation defines that the input interval is totally bad.
def model_3_property_check_is_bad_interval(outputMin, outputMax):
    if outputMax < 0.5:
        return True
    else:
        return False

In [12]:
def model_3_find_all_counter_intervals_examples(nn_model, interval_list):
    # center_point_list = []
    counter_interval_list = []
    counter_example_list = []
    uncertain_interval_list = []
    for interval in interval_list:
        center_point = find_center_point(interval)
        prediction = nn_model.predict(center_point)
        if prediction < 0.5:
            counter_example_list.append(center_point)
            counter_interval_list.append(interval)
        else:
            uncertain_interval_list.append(interval)
    return counter_interval_list, counter_example_list, uncertain_interval_list

In [13]:
# the percent of the area that has been certifyied as safe or unsafe.
def model_3_calculate_coverage(result):
    certification_coverage_percent = 0
    area = 0.
    if result["sat"] == 1:
        certification_coverage_percent = 1
        return certification_coverage_percent
    elif (len(result["safety_intervals"])+len(result["counter_intervals"])) >= len(result["uncertain_intervals"]):# the number of intervals in the uncertain list is less
        for uncertain_interval in result["uncertain_intervals"]:
            area += ((uncertain_interval[1][0][0] - uncertain_interval[0][0][0]) * (uncertain_interval[1][0][1] - uncertain_interval[0][0][1]))
        certification_coverage_percent = 1 - area/1600
        return certification_coverage_percent
    else:
        for safety_interval in result["safety_intervals"]:
            area += ((safety_interval[1][0][0] - safety_interval[0][0][0]) * (safety_interval[1][0][1] - safety_interval[0][0][1]))
        for counter_interval in result["counter_intervals"]:
            area += ((counter_interval[1][0][0] - counter_interval[0][0][0]) * (counter_interval[1][0][1] - counter_interval[0][0][1]))
        certification_coverage_percent = area/1600
        return certification_coverage_percent
    
    

In [16]:
def model_3_certification_with_single_input_interval_fifo(orig_input_interval, total_seconds):
    start_time = time.time()
    
    list_fifo = []
    list_fifo.append(orig_input_interval)
    sat = 0
    sat_interval_list = []
    counter_example_list = []
    counter_interval_list = []
    uncertain_interval_list = []
    
    
    while True:
        elapsed_time = time.time() - start_time 
        
        if not list_fifo:
            sat = 1
            break
        
        if elapsed_time > total_seconds:
            print("now searching fifo-todo list."+"in total "+str(len(list_fifo))+" intervals in the fifo-todo-list.")
            counter_interval_list_from_todolist, counter_example_list_from_todolist, uncertain_interval_list = model_3_find_all_counter_intervals_examples(model_3, list_fifo)
            # combine counter examples form the todo list and from the total bad intervals
            for interval in counter_interval_list:
                counter_example_list.append(find_center_point(interval))
            counter_example_list = counter_example_list_from_todolist + counter_example_list
            # combine the counter interval from the todo list and from the previous check
            counter_interval_list = counter_interval_list_from_todolist + counter_interval_list
            break
                
        input_region = list_fifo.pop(0)
        input_min = input_region[0]
        input_max = input_region[1]
        outputMin, outputMax = simple_interval_propagation_model(model_3, input_min, input_max)
        
        if model_3_property_check(outputMin, outputMax):
            sat_interval_list.append(input_region)
        elif model_3_property_check_is_bad_interval(outputMin, outputMax):
            counter_interval_list.append(input_region)
        else:
            interval1, interval2 = split_2(input_min, input_max)
            list_fifo.append(interval1)
            list_fifo.append(interval2) 
            
    result = {
                "sat": sat,
                "safety_intervals": sat_interval_list, # the intervals that satisfy the safety property
                "counter_intervals": counter_interval_list, # the intervals that definitely contain point that violate the safety property
                "counter_examples": counter_example_list, # the points that violate the safety property
                "uncertain_intervals": uncertain_interval_list #the intervals that uncertain if they contain counter example
            }
    return result

In [ ]:
def model_3_certification_with_single_input_interval_fifo_without_finding_counter_intervals_examples(orig_input_interval, total_seconds):
    start_time = time.time()
    
    list_fifo = []
    list_fifo.append(orig_input_interval)
    sat = 0
    sat_interval_list = []
    counter_example_list = []
    counter_interval_list = []
    uncertain_interval_list = []
    
    
    while True:
        elapsed_time = time.time() - start_time 
        
        if not list_fifo:
            sat = 1
            break
        
        if elapsed_time > total_seconds:
            # the intervals remaining in the fifo-todo list are the uncertain intervals
            uncertain_interval_list = list_fifo
            # get counter examples from the counter_interval_list
            for interval in counter_interval_list:
                counter_example_list.append(find_center_point(interval))
            break
                
        input_region = list_fifo.pop(0)
        input_min = input_region[0]
        input_max = input_region[1]
        outputMin, outputMax = simple_interval_propagation_model(model_3, input_min, input_max)
        
        if model_3_property_check(outputMin, outputMax):
            sat_interval_list.append(input_region)
        elif model_3_property_check_is_bad_interval(outputMin, outputMax):
            counter_interval_list.append(input_region)
        else:
            interval1, interval2 = split_2(input_min, input_max)
            list_fifo.append(interval1)
            list_fifo.append(interval2) 
            
    result = {
                "sat": sat,
                "safety_intervals": sat_interval_list, # the intervals that satisfy the safety property
                "counter_intervals": counter_interval_list, # the intervals that definitely contain point that violate the safety property
                "counter_examples": counter_example_list, # the points that violate the safety property
                "uncertain_intervals": uncertain_interval_list #the intervals that uncertain if they contain counter example
            }
    return result

In [ ]:
def model_3_certification_with_single_input_interval_lifo(orig_input_interval, total_seconds):
    start_time = time.time()
    
    list_lifo = []
    list_lifo.append(orig_input_interval)
    sat = 0
    sat_interval_list = []
    counter_example_list = []
    counter_interval_list = []
    uncertain_interval_list = []
    
    
    while True:
        elapsed_time = time.time() - start_time 
        
        if not list_lifo:
            sat = 1
            break
        
        if elapsed_time > total_seconds:
            print("now searching lifo-todo list."+"in total "+str(len(list_lifo))+" intervals in the lifo-todo-list.")
            counter_interval_list_from_todolist, counter_example_list_from_todolist, uncertain_interval_list = model_3_find_all_counter_intervals_examples(model_3, list_lifo)
            # combine counter examples form the todo list and from the total bad intervals
            for interval in counter_interval_list:
                counter_example_list.append(find_center_point(interval))
            counter_example_list = counter_example_list_from_todolist + counter_example_list
            # combine the counter interval from the todo list and from the previous check
            counter_interval_list = counter_interval_list_from_todolist + counter_interval_list
            break
                
        input_region = list_lifo.pop(len(list_lifo)-1)
        input_min = input_region[0]
        input_max = input_region[1]
        outputMin, outputMax = simple_interval_propagation_model(model_3, input_min, input_max)
        
        if model_3_property_check(outputMin, outputMax):
            sat_interval_list.append(input_region)
        elif model_3_property_check_is_bad_interval(outputMin, outputMax):
            counter_interval_list.append(input_region)
        else:
            interval1, interval2 = split_2(input_min, input_max)
            list_lifo.append(interval1)
            list_lifo.append(interval2) 
            
    result = {
                "sat": sat,
                "safety_intervals": sat_interval_list, # the intervals that satisfy the safety property
                "counter_intervals": counter_interval_list, # the intervals that definitely contain point that violate the safety property
                "counter_examples": counter_example_list, # the points that violate the safety property
                "uncertain_intervals": uncertain_interval_list #the intervals that uncertain if they contain counter example
            }
    return result

In [ ]:
def model_3_certification_with_single_input_interval_lifo_without_finding_counter_intervals_examples(orig_input_interval, total_seconds):
    start_time = time.time()
    
    list_lifo = []
    list_lifo.append(orig_input_interval)
    sat = 0
    sat_interval_list = []
    counter_example_list = []
    counter_interval_list = []
    uncertain_interval_list = []
    
    
    while True:
        elapsed_time = time.time() - start_time 
        
        if not list_lifo:
            sat = 1
            break
        
        if elapsed_time > total_seconds:
            # the intervals remaining in the lifo-todo list are the uncertain intervals
            uncertain_interval_list = list_lifo
            # get counter examples from the counter_interval_list
            for interval in counter_interval_list:
                counter_example_list.append(find_center_point(interval))
            break
                
        input_region = list_lifo.pop(len(list_lifo)-1)
        input_min = input_region[0]
        input_max = input_region[1]
        outputMin, outputMax = simple_interval_propagation_model(model_3, input_min, input_max)
        
        if model_3_property_check(outputMin, outputMax):
            sat_interval_list.append(input_region)
        elif model_3_property_check_is_bad_interval(outputMin, outputMax):
            counter_interval_list.append(input_region)
        else:
            interval1, interval2 = split_2(input_min, input_max)
            list_lifo.append(interval1)
            list_lifo.append(interval2) 
            
    result = {
                "sat": sat,
                "safety_intervals": sat_interval_list, # the intervals that satisfy the safety property
                "counter_intervals": counter_interval_list, # the intervals that definitely contain point that violate the safety property
                "counter_examples": counter_example_list, # the points that violate the safety property
                "uncertain_intervals": uncertain_interval_list #the intervals that uncertain if they contain counter example
            }
    return result